# **Phần 1: Giới thiệu về dataset**

CoNLL 2003 là một dataset phổ biến trong lĩnh vực NLP, đặc biệt là Named Entity Recognition (NER).

### 1. Cấu trúc bộ dữ liệu
- Bộ dữ liệu bao gồm 8 tệp chia làm 2 thứ tiếng: tiếng Anh và tiếng Đức.
- Mỗi một ngôn ngữ gồm 1 traning file, 1 development file, 1 test file và 1 file lớn chứa dữ liệu không được gán nhãn.
- Dữ liệu chưa được gán nhãn gồm khoảng 17 triệu token với tiếng Anh và 14 triệu token với tiếng Đức.

### 2. Quá trình xử lý
- Model ban đầu được huấn luyện với training file, sau đó dùng dev data để tinh chỉnh tham số.
- Tuy nhiên, khó khăn là làm sao để có thể kết hợp đồng thời cả dữ liệu chưa được gán nhãn trong quá trình.
- Sự phân chua giữa hai tệp traning và dev cũng đã được điều chỉnh để tránh trường hợp overfitting.

### 3. Nguồn gốc của các mẫu trong dữ liệu
- Với tiếng Anh:
    - Ngữ liệu được lấy từ Reuters Corpus từ tháng 8/1996 đến tháng 8/1997.
    - Traning và dev data lấy trong khoảng 10 ngày cuối tháng 8/1996.
    - Test data lấy từ tháng 12/1996.
    - Phần dữ liệu thô đã qua xử lý lấy từ tháng 9/1996.

- Với tiếng Đức:
    - Ngữ liệu lấy từ ECI Multilingual Text Corpus, đặc biệt là tờ báo Frankfurter Rundschau.
    - Tất cả dữ liệu đều được lấy từ tuần cuối tháng 8/1992.
    - Riêng dữ liệu thô lấy từ tháng 9/1992 đến tháng 12 cùng năm.

### 4. Quá trình tiền xử lý dữ liệu
- Trước khi đưa vào sử dụng, dữ liệu thô trong bộ dữ liệu CoNLL-2003 đã trải qua quá trình tiền xử lí ngôn ngữ nhằm cho mục đích NER, bao gồm tokenization, part-of-speech tagging và chunking.
- POS tagging:
    - Có 2 tokenizer dành riêng cho 2 ngôn ngữ tiếng Anh và tiếng Đức.
    - Tiếng Anh: sử dụng memory-based tagger MBT.
    - Tiếng Đức: dữ liệu được bổ đề (lemma), gắn thẻ và chia nhỏ (chunking) bằng cây quyết định Treetagger.
- Chunking: Quá trình chia nhỏ dữ liệu, thường áp dụng trên các cụm danh từ hay các cụm từ có nhiều hàm nghĩa.
- NER Aniotation:
    - Tất cả các tệp được gắn nhãn của cả hai ngôn ngữ - bao gồm: tranining, dev, test - đều được xử lý hoàn toàn bằng tay tại Đại học Antwerp.
    - Việc gắn nhãn hầu như tuân theo quy ước MUC, ngoài ra còn bổ sung thêm entity MISC để đánh dấu các tên không thuộc những trường còn lại, chủ yếu là các tính từ hay các cụm danh từ, tên sự kiện...

# **Phần 2: EDA**

# A. Set up

## 1. Cài đặt các thư viện và hàm

### 1.1 Các thư viện cần thiết

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns

### 1.2 Các hàm hỗ trợ

In [ ]:
#Đọc các câu
def load_sentences(filepath):

    final = []
    sentences = []

    with open(filepath, 'r') as f:
        
        for line in f.readlines():
            #Kiểm tra có phải là ký hiệu xuống dòng hoặc dòng đầu của file(-DOCSTART- -X- -X- O) không
            if (line == ('-DOCSTART- -X- -X- O\n') or line == '\n'):
                #Nếu trước đó có đọc được các từ trong 1 sentence thì mới append vô và reset lại sentence
                if len(sentences) > 0:
                    final.append(sentences)
                    sentences = []
            #Còn không thì đọc tiếp vào sentence
            else:
                l = line.split(' ')
                sentences.append((l[0], l[3].strip('\n')))
    
    return final

In [ ]:
#Đếm số lượng câu
def num_sentence(data):
    num_sentences = len(data)
    print("Số câu (sentence) trong tập dữ liệu là:", num_sentences)

In [ ]:
def statistic_sentence(data):
    max_sentence_length = 0
    min_sentence_length = float('inf')
    total_sentence_length = 0
    for sentence in data:
        sentence_length = len(sentence)
        total_sentence_length += sentence_length
        if sentence_length > max_sentence_length:
            max_sentence_length = sentence_length
        if sentence_length < min_sentence_length:
            min_sentence_length = sentence_length
    average_sentence_length = total_sentence_length/len(data)
    
    print("Độ dài câu dài nhất:", max_sentence_length )
    print("Độ dài câu ngắn nhất:", min_sentence_length)
    print("Độ dài trung bình của các câu:", average_sentence_length)

In [ ]:
def count_plot(df, title):
    count_entity = df['Entity'].value_counts()

    fig = px.bar(y=count_entity.values, 
             x=count_entity.index, 
             color = count_entity.index,
             color_discrete_sequence=['#26784A','#74274E'],
             text=count_entity.values,
             title= title,
             template= 'plotly_dark')

    fig.update_traces(width=0.4)
    fig.update_layout(
        xaxis_title="Entity",
        yaxis_title="count",
        font = dict(size=17,family="Franklin Gothic"))
    fig.show()

In [ ]:
def histogram(df, title):
    fig=px.histogram(df,x="Entity" ,title=title, color_discrete_sequence=px.colors.qualitative.Prism)
    fig.update_layout(template="plotly_dark",)
    fig.update_layout(title_font_size=25)
    fig.update_traces(marker=dict(color='blue', opacity=0.7), textposition='outside', texttemplate='%{y}')
    fig.update_layout(
        width=800, 
        height=700 
    )
    fig.show()

In [ ]:
def percentEntity(filepath):
    sentences = load_sentences(filepath)
    total_token_percentages = {}
    total_sentences = len(sentences)
    
    for sentence in sentences:
        token_counts = {}
        total_tokens = 0
        for word in sentence: 
            token = word[1]
            if token in token_counts:
                token_counts[token]+=1
            else:
                token_counts[token]=1
            total_tokens+=1
        token_percentages = {token: count / total_tokens * 100 for token, count in token_counts.items()}
        for token,percentage in token_percentages.items():
            if token in total_token_percentages:
                total_token_percentages[token] += percentage
            else:
                total_token_percentages[token] = percentage
    average_token_percentages = {token: percentage / total_sentences for token, percentage in total_token_percentages.items()}
    df = pd.DataFrame(sorted(list(average_token_percentages.items())), columns=['Entity', '%'])
    df = df.sort_values(by=['%'], ascending=False, ignore_index=True)
    return df

In [ ]:
def pie_chart(df, title):
    a = df['%'][0]
    b = df["%"].iloc[1:19].sum()
    data = [
        {'Entity': 'O', '%': a},
        {'Entity': 'Other', '%': b}
    ]
    df = pd.DataFrame(data)
    fig = px.pie(values=df["%"], 
                 names=df["Entity"], 
                 color_discrete_sequence=["#CDACD1", "#9FD0A8"],
                 title= title,template='plotly_dark')
    fig.update_layout(
    width=800, 
    height=500  
    )

    fig.data[0].marker.line.width = 2
    fig.data[0].marker.line.color='gray'
    fig.update_layout(
        font=dict(size=20,family="Franklin Gothic"))
    fig.show()

## 2. Đọc các tập dữ liệu

### 2.1 Tập train

In [ ]:
train_path = "CoNLL2003/train.txt"
train_sentence = load_sentences(train_path)
columns = ['Token', 'Entity']
token_entity_train = pd.DataFrame(columns=columns)
for sentence in train_sentence:
    temp_df = pd.DataFrame(sentence,columns=columns)
    token_entity_train = pd.concat([token_entity_train,temp_df], axis=0)


In [ ]:
# Loại bỏ các ký tự thừa (dấu ))
token_entity_train_clear = token_entity_train.loc[~((token_entity_train['Token'] == ',') 
                  | (['Token'] == '.') | (token_entity_train['Token'] == '"') 
                  | (token_entity_train['Token'] == '(') | (token_entity_train['Token'] == ')')
                  | ((token_entity_train['Token'] == ':')))]

### 2.2 Tập test

In [ ]:
test_path = "CoNLL2003/test.txt"
test_sentence = load_sentences(test_path)
columns = ['Token', 'Entity']
token_entity_test = pd.DataFrame(columns=columns)
for sentence in test_sentence:
    temp_df = pd.DataFrame(sentence,columns=columns)
    token_entity_test = pd.concat([token_entity_test,temp_df], axis=0)

In [ ]:
# Loại bỏ các ký tự thừa (dấu ))
token_entity_test_clear = token_entity_test.loc[~((token_entity_test['Token'] == ',') 
                  | (['Token'] == '.') | (token_entity_test['Token'] == '"') 
                  | (token_entity_test['Token'] == '(') | (token_entity_test['Token'] == ')')
                  | ((token_entity_test['Token'] == ':')))]

### 2.2 Tập validation

In [ ]:
val_path = "CoNLL2003/valid.txt"
val_sentence = load_sentences(val_path)
columns = ['Token', 'Entity']
token_entity_val = pd.DataFrame(columns=columns)
for sentence in val_sentence:
    temp_df = pd.DataFrame(sentence,columns=columns)
    token_entity_val = pd.concat([token_entity_val,temp_df], axis=0)

In [ ]:
# Loại bỏ các ký tự thừa (dấu ))
token_entity_val_clear = token_entity_val.loc[~((token_entity_val['Token'] == ',') 
                  | (['Token'] == '.') | (token_entity_val['Token'] == '"') 
                  | (token_entity_val['Token'] == '(') | (token_entity_val['Token'] == ')')
                  | ((token_entity_val['Token'] == ':')))]

# B. EDA tập train

## 1. Sentence

### 1.1 Số lượng sentences

In [ ]:
num_sentence(train_sentence)

### 1.2 Các giá trị thống kê cơ bản

In [ ]:
statistic_sentence(train_sentence)

## 2. Entity

### 2.1 Tổng quan các entity

#### 2.1.1 Số lượng các token được gán I-ENTITY, B-ENTITY và O-ENTITY

In [ ]:
print("Số lượng các token được gắn nhãn entity (O, B, I): ",token_entity_train.shape[0])

In [ ]:
histogram(token_entity_train, "Count Of Entities (O, B, I)")

#### 2.1.2 Các entity chính

In [ ]:
entities = token_entity_train_clear.loc[~(token_entity_train_clear['Entity'] == 'O')]

In [ ]:
print("Số lượng các token được gắn nhãn entity (B, I): ",entities.shape[0])

In [ ]:
entities.head()

In [ ]:
entities["Entity"].unique()

In [ ]:
histogram(entities, "Count Of Entities")

**Gộp chung B-Entity và I-Entity thành Entity**

In [ ]:
entities_merge = entities.copy()
entities_merge = entities_merge.loc[entities_merge['Entity'].str.startswith('B-', na=False)]
entities_merge['Entity'] = entities_merge['Entity'].str[2:]
entities_merge.head()

In [ ]:
histogram(entities_merge, "Count Of Entities")

**Nhận xét:**
- Nhìn chung các Entity xuất hiện khá đều nhau, duy nhất mỗi entity MISC có số lần xuất hiện thấp hơn các entity còn lại.

#### 2.1.3 Tỉ lệ trung bình của các từ thuộc nhãn O và nhãn khác O trong một câu 

In [ ]:
df = percentEntity(train_path)
cm = sns.light_palette("green", as_cmap=True)
styled_df = df.style.background_gradient(cmap=cm, subset=['%'])
styled_df

In [ ]:
pie_chart(df, "Average ratio of O-Entity vs other Entity")

### 2.2 Phân tích cụ thể các entity

#### 2.2.1 ORG

In [ ]:
organization = token_entity_train.loc[(token_entity_train['Entity'] == 'B-ORG') | (token_entity_train['Entity'] == 'I-ORG')]

In [ ]:
organization.head()

In [ ]:
count_plot(organization,'ORGANIZATION')

**Nhận xét:**
- Ở entity này, số lượng B_ORG gần như gấp đôi I_ORG
- Nguyên nhân là vì trong hai ngôn ngữ tiếng Anh và tiếng Đức, tên của các tổ chức ngắn hơn và ngoài việc viết đầy đủ thì cách phổ biến hơn là viết tắt bằng cách lấy các chữ cái đầu.
- Ví dụ:
    - European Union -> EU
    - United Nations -> U.N.

#### 2.2.2 PER

In [ ]:
person = token_entity_train.loc[(token_entity_train['Entity'] == 'B-PER') | (token_entity_train['Entity'] == 'I-PER')].reset_index()

In [ ]:
person.head()

In [ ]:
count_plot(person,'PERSON')

**Nhận xét:**
- Ở entity PER, B-PER cao hơn đáng kể so với I-PER vì ngoài cách viết tên đầy đủ, thông thường dùng họ kèm theo các tiền tố như chức vị hay Mr./Mrs...

#### 2.2.3 LOC

In [ ]:
location = token_entity_train.loc[(token_entity_train['Entity'] == 'B-LOC') | (token_entity_train['Entity'] == 'I-LOC')]

In [ ]:
location.head()

In [ ]:
count_plot(location,'LOCATION')

**Nhận xét:**
- Ở entity LOC, B-LOC cao hơn gấp 7 lần I-LOC.
- Nguyên nhân khá rõ ràng, tên địa danh vùng miền trong 2 ngôn ngữ này phần lớn chỉ có 1 từ hoặc chỉ được nhắc đến ngắn gọn.
- Ví dụ:
    - Iraq, Chicago, Texas
    - White House, West Virginia, Wall Street
    - State of Ohio -> Ohio

#### 2.2.4 MISC

In [ ]:
miscellaneous = token_entity_train.loc[(token_entity_train['Entity'] == 'B-MISC') | (token_entity_train['Entity'] == 'I-MISC')]

In [ ]:
miscellaneous.head()

In [ ]:
count_plot(miscellaneous,'MISCELLANEOUS')

**Nhận xét:**
- Phân loại MISC chủ yếu bao gồm các tính từ, đặc biệt là các tính từ chỉ quốc tịch, phần còn lại là các cụm danh từ.
- Một số ví dụ điển hình cho hai nhóm trên:
    - Italian, Japanese, European
    - Financial Times-Stock Exchange, First Battle of the Newbury bypass


# C. EDA tập test

## 1. Sentence

### 1.1 Số lượng sentences

In [ ]:
num_sentence(test_sentence)

### 1.2 Các giá trị thống kê cơ bản

In [ ]:
statistic_sentence(test_sentence)

## 2. Entity

### 2.1 Tổng quan các entity

#### 2.1.1 Số lượng các token được gán I-ENTITY, B-ENTITY và O-ENTITY

In [ ]:
print("Số lượng các token được gắn nhãn entity (O, B, I): ",token_entity_test.shape[0])

In [ ]:
histogram(token_entity_test, "Count Of Entities (O, B, I)")

#### 2.1.2 Các entity chính

In [ ]:
entities = token_entity_test_clear.loc[~(token_entity_test_clear['Entity'] == 'O')]

In [ ]:
print("Số lượng các token được gắn nhãn entity (B, I): ",entities.shape[0])

In [ ]:
entities.head()

In [ ]:
entities["Entity"].unique()

In [ ]:
histogram(entities, "Count Of Entities")

**Gộp chung B-Entity và I-Entity thành Entity**

In [ ]:
entities_merge = entities.copy()
entities_merge = entities_merge.loc[entities_merge['Entity'].str.startswith('B-', na=False)]
entities_merge['Entity'] = entities_merge['Entity'].str[2:]
entities_merge.head()

In [ ]:
histogram(entities_merge, "Count Of Entities")

**Nhận xét:**
- Nhìn chung các Entity xuất hiện khá đều nhau, duy nhất mỗi entity MISC có số lần xuất hiện thấp hơn các entity còn lại.

#### 2.1.3 Tỉ lệ trung bình của các từ thuộc nhãn O và nhãn khác O trong một câu 

In [ ]:
df = percentEntity(test_path)
cm = sns.light_palette("green", as_cmap=True)
styled_df = df.style.background_gradient(cmap=cm, subset=['%'])
styled_df

In [ ]:
pie_chart(df, "Average ratio of O-Entity vs other Entity")

### 2.2 Phân tích cụ thể các entity

#### 2.2.1 ORG

In [ ]:
organization = token_entity_test.loc[(token_entity_test['Entity'] == 'B-ORG') | (token_entity_test['Entity'] == 'I-ORG')]

In [ ]:
organization.head()

In [ ]:
count_plot(organization,'ORGANIZATION')

#### 2.2.2 PER

In [ ]:
person = token_entity_test.loc[(token_entity_test['Entity'] == 'B-PER') | (token_entity_test['Entity'] == 'I-PER')].reset_index()

In [ ]:
person.head()

In [ ]:
count_plot(person,'PERSON')

#### 2.2.3 LOC

In [ ]:
location = token_entity_test.loc[(token_entity_test['Entity'] == 'B-LOC') | (token_entity_test['Entity'] == 'I-LOC')]

In [ ]:
location.head()

In [ ]:
count_plot(location,'LOCATION')

#### 2.2.4 MISC

In [ ]:
miscellaneous = token_entity_test.loc[(token_entity_test['Entity'] == 'B-MISC') | (token_entity_test['Entity'] == 'I-MISC')]

In [ ]:
miscellaneous.head()

In [ ]:
count_plot(miscellaneous,'MISCELLANEOUS')

# D. EDA tập validation

## 1. Sentence

### 1.1 Số lượng sentences

In [ ]:
num_sentence(val_sentence)

### 1.2 Các giá trị thống kê cơ bản

In [ ]:
statistic_sentence(val_sentence)

## 2. Entity

### 2.1 Tổng quan các entity

#### 2.1.1 Số lượng các token được gán I-ENTITY, B-ENTITY và O-ENTITY

In [ ]:
print("Số lượng các token được gắn nhãn entity (O, B, I): ",token_entity_val.shape[0])

In [ ]:
histogram(token_entity_train, "Count Of Entities (O, B, I)")

#### 2.1.2 Các entity chính

In [ ]:
entities = token_entity_val_clear.loc[~(token_entity_val_clear['Entity'] == 'O')]

In [ ]:
print("Số lượng các token được gắn nhãn entity (B, I): ",entities.shape[0])

In [ ]:
entities.head()

In [ ]:
entities["Entity"].unique()

In [ ]:
histogram(entities, "Count Of Entities")

**Gộp chung B-Entity và I-Entity thành Entity**

In [ ]:
entities_merge = entities.copy()
entities_merge = entities_merge.loc[entities_merge['Entity'].str.startswith('B-', na=False)]
entities_merge['Entity'] = entities_merge['Entity'].str[2:]
entities_merge.head()

In [ ]:
histogram(entities_merge, "Count Of Entities")

#### 2.1.3 Tỉ lệ trung bình của các từ thuộc nhãn O và nhãn khác O trong một câu 

In [ ]:
df = percentEntity(test_path)
cm = sns.light_palette("green", as_cmap=True)
styled_df = df.style.background_gradient(cmap=cm, subset=['%'])
styled_df

In [ ]:
pie_chart(df, "Average ratio of O-Entity vs other Entity")

### 2.2 Phân tích cụ thể các entity

#### 2.2.1 ORG

In [ ]:
organization = token_entity_val.loc[(token_entity_val['Entity'] == 'B-ORG') | (token_entity_val['Entity'] == 'I-ORG')]

In [ ]:
organization.head()

In [ ]:
count_plot(organization,'ORGANIZATION')

#### 2.2.2 PER

In [ ]:
person = token_entity_val.loc[(token_entity_val['Entity'] == 'B-PER') | (token_entity_val['Entity'] == 'I-PER')].reset_index()

In [ ]:
person.head()

In [ ]:
count_plot(person,'PERSON')

#### 2.2.3 LOC

In [ ]:
location = token_entity_val.loc[(token_entity_val['Entity'] == 'B-LOC') | (token_entity_val['Entity'] == 'I-LOC')]

In [ ]:
location.head()

In [ ]:
count_plot(location,'LOCATION')

#### 2.2.4 MISC

In [ ]:
miscellaneous = token_entity_val.loc[(token_entity_val['Entity'] == 'B-MISC') | (token_entity_val['Entity'] == 'I-MISC')]

In [ ]:
miscellaneous.head()

In [ ]:
count_plot(miscellaneous,'MISCELLANEOUS')

# E. Nhận xét chung
- Hầu hết các thực thể đều được gắn nhãn phù hợp.
- Tỉ lệ train:test:val = 14:3:3.
- Ưu điểm:
    - Dataset tuân theo cách đánh giá tiêu chuẩn, dễ dàng so sánh giữa các mô hình NER khác.
    - Bộ dữ liệu lấy từ thực tế, phù hợp với ngữ cảnh sử dụng.
- Nhược điểm:
    - Bộ ngữ liệu quá cũ, đều lấy từ trước năm 2000. Ở thời điểm hiện tại, tổ chức ngôn ngữ ít nhiều thay đổi do đó có khó khăn nhất định nếu sử dụng để phân tích ngôn ngữ hiện đại.
    - Nguồn dữ liệu hạn chế do lấy từ các bài báo, văn phong và hình thức văn bản bị bó buộc.
